In [135]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import statsmodels.api as sm
import statsmodels.formula.api as smf
import numpy as np
from collections import Counter
import re
import plotly.express as px
import datetime

In [2]:
arcom_df = pd.read_csv("/Users/thibaultrolland/Projects/arcom/data/ARCOM.csv")

/var/folders/cf/xcc222mj0yg3dzy7mljbx99w0000gn/T/ipykernel_87662/777748067.py:1: DtypeWarning: Columns (37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  arcom_df = pd.read_csv("/Users/thibaultrolland/Projects/arcom/data/ARCOM.csv")


In [3]:
arcom_df.columns

Index(['N° Fiche', 'year', 'week', 'N° Version', 'Annonceur', 'Marque',
       'Produit', 'Date', 'Date VP', 'Média', 'Medium', 'Catégorie', 'Support',
       'Format', 'Secteur', 'Classe', 'Groupe', 'Variété', 'N° Groupe/Variété',
       'Signature', 'Script', 'Incrustation', 'Titre', 'Libellé Version',
       'Visuel', 'Mots Clés', 'Thème', 'Musique / Auteur compositeur',
       'Musique / Titre', 'Agence', 'Réalisateur', 'Producteur', 'Récompense',
       'Musique / Version', 'Type de campagne', 'Musique / Interprète',
       'Partenaires', 'Titre émission', 'Hashtag', 'clean_script',
       'mots_clefs_themes', 'mots_clefs_themes_str', 'visuel_str',
       'visuel_clean', 'visuel_clean_bigram', 'clean_bigram_total', 'date_iso',
       'month_nb'],
      dtype='object')

In [4]:
arcom_df['clean_bigram_total']

0        canard bain incomprehension se_moquer onomatop...
1        fleur_d'oranger savon_noir lessive_liquide eff...
2        camera_cachee interrogation humour_autre noctu...
3        depot leader comparaison odorat recommande tes...
4        oiseau mesange couper_la_parole sachets_de_gra...
                               ...                        
50668    reunions enfant innovation lien_paternel unive...
50669    égo facilité durée_limitée généreux souvenirs ...
50670    casque_vr entrepot efficacite choix innovation...
50671    ascenceur cerveau uterus dent objet_personnifi...
50672    entraineur choisir truspilot equipe selectionn...
Name: clean_bigram_total, Length: 50673, dtype: object

In [145]:
def clean_themes(themes):
    if type(themes) == str:
        themes = themes.split(',')
        themes = [theme.strip() for theme in themes]
        themes = [theme.lower() for theme in themes]
        return themes
    return []

arcom_df['clean_theme'] = arcom_df['Thème'].apply(clean_themes)
arcom_df['clean_date'] = [datetime.datetime.strptime(x, '%d/%m/%Y').date() for x in arcom_df['Date']]

In [146]:
most_frequent_themes = Counter([word for sublist in arcom_df['clean_theme'] for word in sublist])
most_frequent_themes.most_common()

[('pratique', 10319),
 ('interpellation', 10172),
 ('lien media', 9117),
 ('media internet', 8658),
 ('typographie', 8393),
 ('humour autre', 6660),
 ('efficacite', 6283),
 ('choix', 6017),
 ('gourmandise', 5795),
 ('nouveaute autre', 5713),
 ('effet visuel', 5689),
 ('illustration visuelle', 5673),
 ('joie', 5582),
 ('bonne humeur', 5498),
 ('personne celebre autre', 5389),
 ('enfant', 5361),
 ('signature sonore', 5290),
 ('anglais', 5173),
 ('interrogation', 5099),
 ('discours prix', 5086),
 ('gout', 4619),
 ('univers professionnel', 4616),
 ('service autre', 4381),
 ('musique celebre', 4308),
 ('comportement autre', 4277),
 ('couleur', 4142),
 ('animal autre', 4134),
 ('couple', 4098),
 ('musique autre', 3969),
 ('femme', 3962),
 ('quotidien', 3913),
 ('conseil', 3883),
 ('partenariat autre', 3807),
 ('famille', 3764),
 ('signe de qualite autre', 3511),
 ('innovation', 3473),
 ('complicite', 3458),
 ('etonnement', 3410),
 ('valorisation', 3400),
 ('univers urbain', 3390),
 ('signatu

# plot

In [147]:
arcom_df.columns

Index(['N° Fiche', 'year', 'week', 'N° Version', 'Annonceur', 'Marque',
       'Produit', 'Date', 'Date VP', 'Média', 'Medium', 'Catégorie', 'Support',
       'Format', 'Secteur', 'Classe', 'Groupe', 'Variété', 'N° Groupe/Variété',
       'Signature', 'Script', 'Incrustation', 'Titre', 'Libellé Version',
       'Visuel', 'Mots Clés', 'Thème', 'Musique / Auteur compositeur',
       'Musique / Titre', 'Agence', 'Réalisateur', 'Producteur', 'Récompense',
       'Musique / Version', 'Type de campagne', 'Musique / Interprète',
       'Partenaires', 'Titre émission', 'Hashtag', 'clean_script',
       'mots_clefs_themes', 'mots_clefs_themes_str', 'visuel_str',
       'visuel_clean', 'visuel_clean_bigram', 'clean_bigram_total', 'date_iso',
       'month_nb', 'clean_theme', 'group', 'clean_date'],
      dtype='object')

In [158]:
theme = 'nutri-score'

sub_arcom_df = arcom_df[[theme in x for x in arcom_df['clean_theme']]]
fig = px.histogram(sub_arcom_df,
            x='clean_date',
            color='Secteur', title=f'Ads with {theme} in themes')
fig.show()

In [160]:
theme = 'anglais'

sub_arcom_df = arcom_df[[theme in x for x in arcom_df['clean_theme']]]
fig = px.histogram(sub_arcom_df,
            x='clean_date',
            color='Secteur', title=f'Ads with {theme} in themes')
fig.show()

# Frequency of thematics

In [149]:
arcom_df['group'] = ['treatment' if x == 'AALIMENTATION' else 'group' for x in arcom_df['Secteur']]

In [150]:
token_pattern = r'\b(?:\w+-\w+|\w+|nutri-score)\b'
vectorizer = CountVectorizer(tokenizer =lambda t: re.findall(token_pattern, t.lower()))

In [151]:


X = vectorizer.fit_transform(arcom_df['Thème'])
word_counts = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
word_counts


/Users/thibaultrolland/anaconda3/envs/arcomenv/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:525: UserWarning:

The parameter 'token_pattern' will not be used since 'tokenizer' is not None'



,2019,2022,2023,24,6,60,a,absurde,accent,acronyme,...,vtt,vue,vulgarite,webserie,windows,yatching,yoga,you,youtubeur,zezaiement
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50668,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
50669,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
50670,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
50671,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [152]:

#nutriscore_date='2017-06-01'

def estimate_DiD(target_word, date = datetime.date(2017, 6, 1)):
    data = pd.concat([arcom_df['clean_theme'], arcom_df['group'], word_counts], axis=1)
    data['outcome'] = data[target_word]
    data['prior'] = np.where(arcom_df['clean_date']<date, 1, 0)
    data['post'] = np.where(arcom_df['clean_date']>=date, 1, 0)
    data['treated'] = np.where(data['group'] == 'treatment', 1, 0)
    data['post_treated'] = data['post'] * data['treated']
    model = smf.ols('outcome ~ post + treated + post_treated', data=data).fit()
    print(model.summary())
    did_estimate = model.params['post_treated']
    print('\r-------------------------\r')
    print(f'DiD Estimate for the word {target_word}: {did_estimate}')
    print('\r-------------------------\r')



In [153]:
estimate_DiD('gout')

                            OLS Regression Results                            
Dep. Variable:                outcome   R-squared:                       0.216
Model:                            OLS   Adj. R-squared:                  0.216
Method:                 Least Squares   F-statistic:                     4652.
Date:                Wed, 10 Jul 2024   Prob (F-statistic):               0.00
Time:                        18:59:41   Log-Likelihood:                -2629.5
No. Observations:               50673   AIC:                             5267.
Df Residuals:                   50669   BIC:                             5302.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        0.0300      0.002     14.098   

In [154]:
estimate_DiD('nutri-score')

                            OLS Regression Results                            
Dep. Variable:                outcome   R-squared:                       0.016
Model:                            OLS   Adj. R-squared:                  0.016
Method:                 Least Squares   F-statistic:                     272.3
Date:                Wed, 10 Jul 2024   Prob (F-statistic):          2.33e-175
Time:                        18:59:42   Log-Likelihood:                 80498.
No. Observations:               50673   AIC:                        -1.610e+05
Df Residuals:                   50669   BIC:                        -1.610e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept     3.591e-15      0.000   8.72e-12   

In [155]:
estimate_DiD('gourmandise')

                            OLS Regression Results                            
Dep. Variable:                outcome   R-squared:                       0.228
Model:                            OLS   Adj. R-squared:                  0.228
Method:                 Least Squares   F-statistic:                     4988.
Date:                Wed, 10 Jul 2024   Prob (F-statistic):               0.00
Time:                        19:00:26   Log-Likelihood:                -7328.9
No. Observations:               50673   AIC:                         1.467e+04
Df Residuals:                   50669   BIC:                         1.470e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        0.0398      0.002     17.053   